# Calcul des coordonnées des coins d'un champ récepteur d'activation

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

In [2]:
def get_receptive_field(pos, k, s, p):
    _to_tuple = lambda v: (v, v)
    if type(k) == int:
        k = _to_tuple(k)
    if type(s) == int:
        s = _to_tuple(s)
    if type(p) == int:
        p = _to_tuple(p)
    column = -p[1] + pos[1] * s[1]
    row = -p[0] + pos[0] * s[0]
    return (row, column), (row+k[1], column+k[0])

In [3]:
def get_receptive_field_conv2d(pos, k, s, p):
    return get_receptive_field(pos, k, s, p)

In [4]:
def get_receptive_field_pool2d(pos, k, s, p): 
    return get_receptive_field(pos, k, s, p)

In [5]:
model = models.alexnet(weights='IMAGENET1K_V1')
model.eval()
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [17]:
pos_a = (20, 20)
idx_module = 3

tlc, brc = pos_a, pos_a
for module in reversed(model.features[:idx_module]):
    print(module)
    if isinstance(module, nn.Conv2d):
        tlc, _ = get_receptive_field_conv2d(tlc, module.kernel_size, module.stride, module.padding)
        _, brc = get_receptive_field_conv2d(brc, module.kernel_size, module.stride, module.padding)
    elif isinstance(module, nn.MaxPool2d):
        tlc, _ = get_receptive_field_pool2d(tlc, module.kernel_size, module.stride, module.padding)
        _, brc = get_receptive_field_pool2d(brc, module.kernel_size, module.stride, module.padding)
    print(tlc, brc)


MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
(40, 40) (43, 43)
ReLU(inplace=True)
(40, 40) (43, 43)
Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
(158, 158) (181, 181)


In [16]:
input_size = torch.Size([1, 3, 224, 224])
receptive_field = (
    (max(0, tlc[0]), max(0, tlc[1])),
    (min(input_size[2], brc[0]), min(input_size[3], brc[1]))
)
print(receptive_field)


((158, 158), (181, 181))
